In [1]:
import pandas as pd
import psycopg2
import getpass

In [2]:
user = 'postgres'
host = '0.0.0.0'
dbname = 'mimic'
password = "postgres"
schema = '1.4'

In [3]:
# Connect to the database, assuming the database is connected via "docker start demo-mimic-container"

con = psycopg2.connect(dbname=dbname, user=user, host=host, password = password,  port = 5433)
query_schema = 'set search_path to mimiciii;'


In [4]:
def sql_vital_sign_query(vital_sign, connection):
    """ 
    Query MIMIC for a specific vital sign defined as 'vital_sign'. The database 
    is accessed through 'connection'. Return a dataframe with the query result.
    """

    vital_sign_query = """
   SELECT de.icustay_id
     , de.subject_id
     , ie.intime
     , de.charttime
     , di.label
     , de.value
     , de.valuenum
     , de.valueuom
     , pt.dob
     , ie.hadm_id
     , de.itemid
     , ad.hospital_expire_flag
     
   FROM chartevents de
   INNER join d_items di
   ON de.itemid = di.itemid
   INNER join icustays ie
   ON de.icustay_id = ie.icustay_id
   INNER join admissions ad
   ON ie.hadm_id = ad.hadm_id
   INNER join patients pt
   ON de.subject_id = pt.subject_id
   where di.label = '{}'
   """.format(str(vital_sign))

    query = vital_sign_query
    vital_sign_data_frame = pd.read_sql_query(query, connection)

    return vital_sign_data_frame

In [5]:
query = """
SELECT ad.subject_id
    , ad.ethnicity
    , co.callout_outcome

   FROM admissions ad
   INNER join callout co
   ON ad.subject_id = co.subject_id
"""

df = pd.read_sql_query(query_schema + query, con)

df.head()

,subject_id,ethnicity,callout_outcome
0,10017,WHITE,Discharged
1,10026,WHITE,Discharged
2,10027,WHITE,Discharged
3,10029,WHITE,Discharged
4,10033,WHITE,Discharged


In [6]:
systolic_BP_df = sql_vital_sign_query(
    'Non Invasive Blood Pressure systolic', con)

In [7]:
# Check how the data looks
print("We have", len(systolic_BP_df), "blood pressure measurements.")
systolic_BP_df.head()

We have 4884 blood pressure measurements.


,icustay_id,subject_id,intime,charttime,label,value,valuenum,valueuom,dob,hadm_id,itemid,hospital_expire_flag
0,285750,40177,2169-05-11 20:45:56,2169-05-11 21:00:00,Non Invasive Blood Pressure systolic,92,92.0,mmHg,2082-06-27,198480,220179,1
1,285750,40177,2169-05-11 20:45:56,2169-05-11 22:00:00,Non Invasive Blood Pressure systolic,115,115.0,mmHg,2082-06-27,198480,220179,1
2,285750,40177,2169-05-11 20:45:56,2169-05-11 23:00:00,Non Invasive Blood Pressure systolic,111,111.0,mmHg,2082-06-27,198480,220179,1
3,285750,40177,2169-05-11 20:45:56,2169-05-12 03:30:00,Non Invasive Blood Pressure systolic,118,118.0,mmHg,2082-06-27,198480,220179,1
4,285750,40177,2169-05-11 20:45:56,2169-05-12 04:00:00,Non Invasive Blood Pressure systolic,104,104.0,mmHg,2082-06-27,198480,220179,1
